Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# MTH793P - Coursework 6-2

This is a template notebook for the computational exercises of Coursework 6, part **2/2** of the module MTH793P, Advanced machine learning. Closely follow the instructions in this template in order to complete the assessment and to obtain full marks. For the submitted notebook, please only modify cells where you are instructed to do so. Failure to comply may result in unexpected errors that can lead to mark deductions.

In this part we will implement and explore PCA.

**<font color='red'>IMPORTANT NOTE:</font>**<br>
When using **svd** in this part, always pass the parameter: **<font color='red'>full_matrices=False</font>**.
This is important to speed up the computations.


In [ ]:
### 
### Required imports
###

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from numpy.linalg import svd
from scipy import io

from numpy.testing import assert_almost_equal

%matplotlib inline

## PCA for dimension reduction
We will first demonstrate PCA on a 13-dimensional dataset, by loading wine dataset from sklearn, see info [here](https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-dataset). <br>
This dataset contains chemical analysis of N=178 different wines by three different cultivators.
<br>
The analysis contains the folowing measurements:
<dd class="field-odd"><ul class="simple">
<li><p>Alcohol</p></li>
<li><p>Malic acid</p></li>
<li><p>Ash</p></li>
<li><p>Alcalinity of ash</p></li>
<li><p>Magnesium</p></li>
<li><p>Total phenols</p></li>
<li><p>Flavanoids</p></li>
<li><p>Nonflavanoid phenols</p></li>
<li><p>Proanthocyanins</p></li>
<li><p>Colour intensity</p></li>
<li><p>Hue</p></li>
<li><p>OD280/OD315 of diluted wines</p></li>
<li><p>Proline</p></li>
</ul>
</dd>
</dl>

So overall, we have N=178 data points, lying in $\mathbb{R}^{D}$, with D=13. We stack all points together into a matrix **<font color='red'>X_wine</font>** $\in \mathbb{R}^{D\times N}$.<br>
We have labels 0,1, or 2 for each wine (clutivator). The true labels are given in **<font color='red'>L_wine</font>**.<br>
We want to see whether PCA can be helpful in the unsupervised task of clustering the 178 wines.

We start by loading the dataset, and printing the first 5 data points, just to get a general impression.

In [ ]:
X_wine, L_wine = load_wine(return_X_y=True)
X_wine = X_wine.T
np.set_printoptions(suppress=True)

print('First 5 data points:')
print('--------------------')
print(X_wine[:,0:5])

Write a function called **<font color='red'>pc_transform</font>**. The inputs of this functions are:
* **<font color='red'>X</font>** - the data matrix ($D\times N$)
* **<font color='red'>PCS</font>** - a list of indexes for the principal components we want to use.
The function should compute the principal components listed in **PCS**, and then return the projection of **X** on these principal components.<br>

For example, **get_transform(X, \[0,4\])** should return the projection of **X** on the **first** and **fifth** principal components.

In [ ]:
def pc_transform(X, PCS):
# YOUR CODE HERE
raise NotImplementedError()

The next box contains a sanity check to see that your function works properly.

In [ ]:
X_test = np.array([[1,4,3,2],[4,6,7,3],[2,0,7,1]])
U_test = pc_transform(X_test, [0,2])
RES = np.array([[ -4.49780587,  -6.27877721, -10.02650226,  -3.61725016],
 [ -0.87620088,   0.09313905,   0.24070169,   0.26063614]])

assert_almost_equal(U_test, RES)

### DO NOT EDIT THIS CELL ###

Compute the projection of **X_wine** along the first 2 principal components.
Place the result in a matrix **<font color='red'>Y</font>** $\in \mathbb{R}^{2\times N}$. <br>
We then plot the projected points, and colour them according to the original labels.<br>

**<font color='blue'>Q:</font>** What do you think, is this projection helpful for clustering?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
plt.scatter(Y[0,:], Y[1,:], c=L_wine);

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

We will now try to improve the result by normalisation.<br>
The idea is that we take each row of **X_wine**, and standartise it to have mean 0 and variance 1.<br>

**<font color='blue'>Q:</font>** Why might we want to do that? (HINT: take a look at the columns printed above).<br><br>
You man use the class **StandardScaler** from **sklearn.preprocessing** to do so (already imported). <br>
Place the standartised version of **X_wine** in a matrix called **<font color='red'>XS</font>**. Then compute the first 2 principal componets for **XS**, and place the result into **<font color='red'>YS</font>**. <br>
**NOTE:** Make sure you are normalising the **rows** and not the **columns**

**<font color='blue'>Q:</font>** Is this a better representation for the data?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

plt.scatter(YS[0,:], YS[1,:], c=L_wine);

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

## PCA for eigenfaces

In this part, we work with a dataset consisting of numerous faces and create a basis of so-called eigenfaces.<br>
The images are 192x168, and are stored as vectors of length 32,256.

This utility function will just let us convert an image from vector to matrix representation, so it can be showed on the screen.

In [ ]:
def vec2img(vec):
    return np.reshape(vec,(168,192)).T

We load the faces database.<br>
**<font color='red'>FACES</font>** - a where each entry is a collection of images of a single person <br>
**<font color='red'>NPEOPLE</font>** - number of people in the list (should be 20)<br>
**<font color='red'>NFACES</font>** - number of images per person (should be 64)

In [ ]:
f = open('faces.npy','rb')
FACES = np.load(f)
NPEOPLE = len(FACES)
NFACES = 64
NR = int(np.sqrt(NFACES))
f.close()

First, we will examine the photos of a **single** person.<br>
**<font color='red'>PI</font>** - the index of the person we examine. <br>
**<font color='red'>X_person</font>** - the data matrix, containing all images of person PI, as columns.<br>

You can change **PI** as you wish, to experiment with the photos of different people.

We start by presenting all the photos. 

In [ ]:
PI = 1

In [ ]:
X_person =  FACES[PI]
         
plt.figure(figsize=(15,15))
for i in range(NFACES):
    im = vec2img(X_person[:,i])
    plt.subplot(NR,NR,i+1)
    plt.imshow(im,cmap='gray')
    plt.axis('off')

Next, we want to find the "**eigenfaces**", i.e., the principal components of this collection of images.<br>
When using PCA, we should do all the processing for the **cetnered** data, i.e., with mean 0.
Therefore, take the following steps:
1. Define **<font color='red'>M_person</font>** to be the mean of all images in **X_person** (should be a 32,256-dimensional vector).
1. Subtract **M_person** from all images. Place the result in **<font color='red'>XZ_person</font>**.
1. Find the left-singular vectors of **XZ_person** (i.e., the matrix U in the SVD). Place the result in **<font color='red'>U_person</font>**.

At the end of this process, the columns of **U_person** should represent the **eigenfaces**.<br>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

We will present the mean face **M_person** along with the first 15 eigenfaces in **U_person**.

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(4,4,1)
plt.imshow(vec2img(M_person), cmap='gray')
plt.axis('off')
plt.title('mean')

for i in range(15):
    plt.subplot(4,4,i+2)
    plt.imshow(vec2img(U_person[:,i]), cmap='gray')
    plt.axis('off')
    plt.title('$u_{%i}$' % (i+1))

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

Next, we want to try to get a feeling for what some of the eigenfaces represent.<br>
We start by exploring the role of the first 2 eigenfaces.<br>
To do so, we compute the reconstruction of **XZ_person** based on the first two principal components. We then add back the mean **M_person**.<br> The result should be placed in **<font color='red'>X12</font>**.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

We present the results for all the images. <br>
**<font color='blue'>Q:</font>** Can you notice the effect of these two eigenfaces? 

In [ ]:
plt.figure(figsize=(15,15))
for i in range(NFACES):
    im = vec2img(X12[:,i])
    plt.subplot(NR,NR,i+1)
    plt.imshow(im,cmap='gray')
    plt.axis('off')

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

Next, we will do the same but for the 3rd and 4th eigenfaces. Place the result in **<font color='red'>X34</font>**.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**<font color='blue'>Q:</font>** Can you notice the effect of these two eigenfaces? Is it the same or different than the previous two?

In [ ]:
plt.figure(figsize=(15,15))
for i in range(NFACES):
    im = vec2img(X34[:,i])
    plt.subplot(NR,NR,i+1)
    plt.imshow(im,cmap='gray')
    plt.axis('off')

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

Next, we will have a similar exercise, but instead of taking a single person, we will take **all** of them (together).

We place all the images in **<font color='red'>X_all</font>** and show a random collection of them.

In [ ]:
X_all = np.concatenate(FACES,1)
         
plt.figure(figsize=(15,15))
for i in range(NFACES):
    j = np.random.randint(NFACES*NPEOPLE)
    im = vec2img(X_all[:,j])
    plt.subplot(NR,NR,i+1)
    plt.imshow(im,cmap='gray')
    plt.axis('off')

Find and present the eigenfaces. This step is similar to what we did for a single person.<br>
Here, in addition to storing the singular vectors in **<font color='red'>U_all</font>**, you should also save the **singular values**. <br>
Make sure the singular values are stored in **<font color='red'>S_all</font>**.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(4,4,1)
plt.imshow(vec2img(M_all), cmap='gray')
plt.axis('off')
plt.title('mean')

for i in range(15):
    plt.subplot(4,4,i+2)
    plt.imshow(vec2img(U_all[:,i]), cmap='gray')
    plt.axis('off')
    plt.title('$u_{%i}$' % (i+1))

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

In the next box we try to test the effect of the  eigenfaces in different way. <br>
For each eigenface **u<sub>i</sub>** we show the image **M + t$\cdot$u<sub>i</sub>**, where **t** is in $\big[-\frac{\sigma_i}{20},\frac{\sigma_i}{20}\big]$.<br>
The list **<font color='red'>I</font>** indicates which eigenfaces to examine. You can experiment with different indexes.

**<font color='blue'>Q:</font>** Can you notice the effects of the different eigenfaces? 

In [ ]:
I = [0,1,2,3,4,5]
NT = 11
SCALE = 20

In [ ]:
NI = len(I)
M = X_all.mean(1)

plt.figure(figsize=(15,10))
for i in range(NI):
    T = np.linspace(-S_all[I[i]]/SCALE,S_all[I[i]]/SCALE,NT)
    for j in range(NT):
        im = M + T[j]*U_all[:,I[i]]
        plt.subplot(NI,NT, i*NT+j+1)
        plt.imshow(vec2img(im),cmap='gray')
        plt.axis('off')

Finally, we would like to see whether the eigenfaces can be useful for clustering the face images.
We will only focus on **binary** clustering here.

The list in **<font color='red'>PIS</font>** should contain exactly two indexes of the two people you want to test clustering on. <br>
The list in **<font color='red'>PCS</font>** should contain exactly two indexes, for the principal components to be used.<br>


You should create a data matrix **<font color='red'>X_bin</font>** that contains all images from these two people.<br>
Next, use the **<font color='red'>pc_transform</font>** function that you wrote above, to project on the two principal components listed in **PCS**. <br>Place the result in the matrix **<font color='red'>Y_bin</font>**.<br>

These principal compoments will be plotted, coloured according to the person. 

Try to experiment with different people and different PCs.

**<font color='blue'>Q:</font>** Which principal components seem to be separating the images well?

In [ ]:
PIS = [5,15]
PCS = [4,6]

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###

In [ ]:
cols = np.zeros(NFACES*2)
cols[NFACES:] = 1

# YOUR CODE HERE
raise NotImplementedError()

plt.scatter(Y_bin[0,:], Y_bin[1,:],c=cols);

In [ ]:
### DO NOT REMOVE/EDIT THIS CELL ###